In [1]:
import FinanceDataReader as fdr
import pandas as pd
import numpy as np
import csv
import pickle
import html5lib
import requests
import json
from bs4 import BeautifulSoup
import joblib

In [2]:
NASDAQ = fdr.StockListing('NASDAQ')

100%|██████████| 4319/4319 [00:28<00:00, 153.24it/s]


In [17]:
NASDAQ.iloc[9]

Symbol                  PEP
Name            PepsiCo Inc
Industry                 음료
IndustryCode         541010
Name: 9, dtype: object

In [18]:
df_price = pd.DataFrame()
df_price[NASDAQ.loc[9].Name] = fdr.DataReader(NASDAQ.loc[9].Symbol, '1980','2021')['Close'] # Since it is oldest. Start with Pepsi
for i in range(0,9): # Except for stocks which are not existed before 2020 
    df_price[NASDAQ.loc[i].Name] = fdr.DataReader(NASDAQ.loc[i].Symbol, '1980','2021')['Close']
for i in range(10,38): # Except for stocks which are not existed before 2020
    df_price[NASDAQ.loc[i].Name] = fdr.DataReader(NASDAQ.loc[i].Symbol, '1980','2021')['Close']
for i in range(39,78):
    df_price[NASDAQ.loc[i].Name] = fdr.DataReader(NASDAQ.loc[i].Symbol, '1980','2021')['Close']
for i in range(80,87):
    df_price[NASDAQ.loc[i].Name] = fdr.DataReader(NASDAQ.loc[i].Symbol, '1980','2021')['Close']
for i in range(88,95):
    df_price[NASDAQ.loc[i].Name] = fdr.DataReader(NASDAQ.loc[i].Symbol, '1980','2021')['Close']
for i in range(96,105):
    df_price[NASDAQ.loc[i].Name] = fdr.DataReader(NASDAQ.loc[i].Symbol, '1980','2021')['Close']

In [19]:
df_price.shape

(10340, 100)

In [20]:
df_price

,PepsiCo Inc,Apple Inc,Microsoft Corp,Amazon.com Inc,Alphabet Inc Class A,NVIDIA Corp,Alphabet Inc Class C,Meta Platforms Inc,Tesla Inc,Broadcom Inc,...,Cognizant Technology Solutions Corp,Walgreens Boots Alliance Inc,Coca-Cola Europacific Partners PLC,Baker Hughes Co,SBA Communications Corp,Arch Capital Group Ltd,CrowdStrike Holdings Inc,Verisk Analytics Inc,ANSYS Inc,Beigene Ltd ADR
Date,,,,,,,,,,,,,,,,,,,,,
1980-01-02,1.361111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-01-03,1.368056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-01-04,1.381944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-01-07,1.388889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-01-08,1.388889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24,145.059998,131.970001,222.750000,158.634506,86.708000,129.937500,86.942497,267.399994,220.589996,431.459991,...,81.070000,39.619999,47.740002,21.010000,275.019989,34.860001,221.119995,201.229996,364.170013,263.720001
2020-12-28,146.910004,136.690002,224.960007,164.197998,88.697998,129.000000,88.804497,277.000000,221.229996,431.880005,...,81.370003,39.660000,48.320000,20.879999,276.250000,35.330002,212.550003,207.979996,363.720001,252.639999
2020-12-29,147.419998,134.869995,224.149994,166.100006,87.888000,129.432495,87.935997,276.779999,221.996674,429.040009,...,80.500000,39.410000,48.639999,20.750000,276.000000,35.299999,205.929993,205.669998,360.850006,245.000000


In [21]:
df_price_np = df_price.to_numpy()
df_train = np.zeros((251, 732000))
rate = 0
for i in range(0,100):
    for j in range(250,7570): # 7320
        df_train[0:250,rate] = (df_price_np[j-250:j,i] / df_price_np[j, i])
        df_train[250,rate] = (df_price_np[j+21,i] / df_price_np[j,i])
        rate += 1

df_train = df_train.T

df_test = np.zeros((251, 274900))
rate = 0
for j in range(7570,10340-21): #2749
    for i in range(0,100):
        df_test[0:250,rate] = (df_price_np[j-250:j,i] / df_price_np[j, i])
        df_test[250,rate] = (df_price_np[j+21,i] / df_price_np[j,i])
        rate += 1

df_test = df_test.T


df = np.zeros((13000,251)) #100*130
for i in range(0,119):
    df[100*i:100*i+100,:] = df_test[i*100*23:i*100*23+100,:]
df_test = df

np.nan_to_num(df_train, copy=False)
print('done')

arr = []#np.zeros((1,1)).astype(int)

for i in range(0,732000):
    for j in range(0,251):
        if df_train[i,j] == 0:
            #df_train = np.delete(df_train,i, axis= 0)
            arr.append(i)
            continue

arr2 = np.array(arr)
arr2 = np.unique(arr2)
df_train = np.delete(df_train, arr2, axis=0)
#df.to_csv('test_uncertainty.csv', index=False)

df_train2 = np.zeros((df_train.shape[0], 254))
df_train2[:,:251] = df_train[:,:251] 
# 100: 2% increase,010: -2% decrease, 001: none of those

for i in range(len(df_train)):
    if df_train[i,250] >= 1.02:
        df_train2[i,251] = 1
    elif df_train[i,250] <= 0.98:
        df_train2[i,252] = 1
    elif df_train[i,250] >= 0.98 and df_train[i,250] <= 1.02:
        df_train2[i,253] = 1

joblib.dump(df_train2, 'one_hot_train_2%.pkl')
train = joblib.load('one_hot_train_2%.pkl')
train = pd.DataFrame(train)
train.to_csv('train_one_hot_2%.csv',index = False)
print('train file!')

for i in range(len(df_test)):
    for j in range(250):
        if np.isnan(df_test[i,j]):
            df_test[i,:] = np.NaN
            continue
        
print(np.sum(np.isnan(df_test), axis=0))
df_test2 = np.zeros((13000, 254))

df_test2[:,:251] = df_test[:,:251]
# 100: 2% increase,010: -2% decrease, 001: none of those

for i in range(len(df_test)):
    if df_test[i,250] >= 1.02:
        df_test2[i,251] = 1
    elif df_test[i,250] <= 0.98:
        df_test2[i,252] = 1
    elif df_test[i,250] >= 0.98 and df_test[i,250] <= 1.02:
        df_test2[i,253] = 1

df_test2 = pd.DataFrame(df_test2)
joblib.dump(df_test2, "one_hot_test_2%.pkl")
df_test2.to_csv("test_one_hot_2%.csv", index=False)
print('test file!')

done
train file!
[961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961
 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961
 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961
 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961
 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961
 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961
 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961
 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961
 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961
 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961
 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961
 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961
 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961 961
 961 961 961 961 961 961 961 961 9

Finding row of 2010

In [22]:
df_price.loc['2009-12-31']

PepsiCo Inc                 60.799999
Apple Inc                    7.526071
Microsoft Corp              30.480000
Amazon.com Inc               6.726000
Alphabet Inc Class A        15.515015
                              ...    
Arch Capital Group Ltd       7.950000
CrowdStrike Holdings Inc          NaN
Verisk Analytics Inc        30.280001
ANSYS Inc                   43.459999
Beigene Ltd ADR                   NaN
Name: 2009-12-31 00:00:00, Length: 100, dtype: float64

In [23]:
df_price.loc[df_price['PepsiCo Inc'] == 60.799999]

,PepsiCo Inc,Apple Inc,Microsoft Corp,Amazon.com Inc,Alphabet Inc Class A,NVIDIA Corp,Alphabet Inc Class C,Meta Platforms Inc,Tesla Inc,Broadcom Inc,...,Cognizant Technology Solutions Corp,Walgreens Boots Alliance Inc,Coca-Cola Europacific Partners PLC,Baker Hughes Co,SBA Communications Corp,Arch Capital Group Ltd,CrowdStrike Holdings Inc,Verisk Analytics Inc,ANSYS Inc,Beigene Ltd ADR
Date,,,,,,,,,,,,,,,,,,,,,
2009-12-31,60.799999,7.526071,30.48,6.726,15.515015,4.67,15.441621,NaN,NaN,18.290001,...,22.665001,36.720001,21.200001,40.48,34.16,7.95,NaN,30.280001,43.459999,NaN


In [24]:
df_price.iloc[7570]

PepsiCo Inc                 60.799999
Apple Inc                    7.526071
Microsoft Corp              30.480000
Amazon.com Inc               6.726000
Alphabet Inc Class A        15.515015
                              ...    
Arch Capital Group Ltd       7.950000
CrowdStrike Holdings Inc          NaN
Verisk Analytics Inc        30.280001
ANSYS Inc                   43.459999
Beigene Ltd ADR                   NaN
Name: 2009-12-31 00:00:00, Length: 100, dtype: float64